In [6]:
import pandas as pd
from lifelines import CoxPHFitter
from src.helper_methods import *
from src.pipe_store import *
from src.constants import *
from lifelines import CoxPHFitter

%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [7]:
RELEVANT_DESA_BAD = {
    '71TD', '144QL', '70DRA', '80TLR', '76ED', '70DA', '45EV', '67F', '76ET', '163RG', '67VG', 
    '150AAH', '158T', '62QE', '45GV', '45KE', '71A', '66NH', '80TA', '114R', '44KM'
}

# Data Loading

### Total Cohort

In [8]:
path_cohort = '~/UMCUtrecht/KaplanMeier/DESAsurvival.pickle'
path_antibody = '~/UMCUtrecht/20211104_mismatch_ep_db.pickle'
df_abs = pd.read_pickle(path_antibody)
df_cohort = pd.read_pickle(path_cohort)
df = df_abs.merge(df_cohort, on='TransplantID')
cols = [
    'TransplantID', 'DESA_Status', 'EpvsHLA_Donor', '#DESA', 'DESA', 'RecipientAge_NOTR', 
    'DonorAge_NOTR', 'TypeOfDonor_NOTR', 'FailureCode10Y_R', 'GraftSurvival10Y_R', 'IL2rMoAb',
    'CIPHour_DBD', 'CIPHour_DCD' , 'LivingDonorNum', # 'FailureCode10y'
]
df = df[cols]
df.columns

Index(['TransplantID', 'DESA_Status', 'EpvsHLA_Donor', '#DESA', 'DESA',
       'RecipientAge_NOTR', 'DonorAge_NOTR', 'TypeOfDonor_NOTR',
       'FailureCode10Y_R', 'GraftSurvival10Y_R', 'IL2rMoAb', 'CIPHour_DBD',
       'CIPHour_DCD', 'LivingDonorNum'],
      dtype='object')

### All the DESA found

In [9]:
list_set_desa = df[(df.DESA_Status.eq('DESA'))].DESA.values
desas = {ep for desa_set in list_set_desa for ep in desa_set}
print(desas)

{'38A', '80TL', '71QS', '32Y', '77N[DR]', '113HD', '96H', '95V', '97N', '62LQ', '253Q', '67LQ', '26L[DQ]', '13FE', '189S', '130Q', '66IC', '152RR', '65QIA', '70DRA', '70Q', '37FL', '158T', '71ATD', '74Y', '160AD', '67VG', '77TY', '73AN', '65RK', '47KHL', '40E', '180VTP', '77N[A]', '109F', '45EV', '180E', '104AK', '76ET', '70DA', '156DA', '70GT', '9H', '30H[DQ]', '37YV[DR]', '24T', '46VY', '57DE', '96HK', '9T', '66NM', '80K', '108T', '130A', '116L', '30G[B]', '70QQ', '62QE', '166DG', '63NI', '70QA', '32H', '138MI', '116I', '199V', '151AHA', '55PPD', '129QS', '44RMA', '56L', '245V', '69AA', '181M', '97W', '75S', '62EE', '55PPA', '57V[DR]', '116F', '30C', '9F[DQ]', '76ED', '98Q', '180VMP', '45GV', '67VT', '133RS', '150AAH', '52PQ', '71E', '147L', '57V[DQ]', '193PL', '55PP', '163E', '66IY', '245AS', '25R', '76VDT', '125G', '65GK', '26F', '70QT', '73TVS', '31FY', '43R', '37N', '30G[DR]', '95I', '28D', '81ALR', '86A', '44RME', '25Q', '125SQ', '52SK', '99S', '73A', '75IL', '74EL', '144K', '15

### Total DESA

In [10]:
len(desas)

313

### NUmber of Tx with DESA

In [11]:
len(df[(df.DESA_Status.eq('DESA'))])

439

### NUmber of Tx with clinically relevant DESA

In [12]:
print('Total:', df['DESA'].apply(lambda x: bool(x & RELEVANT_DESA_BAD)).sum())
print('Deceased', df[df.TypeOfDonor_NOTR == 'Deceased']['DESA'].apply(lambda x: bool(x & RELEVANT_DESA_BAD)).sum())
print('Living', df[df.TypeOfDonor_NOTR == 'Living']['DESA'].apply(lambda x: bool(x & RELEVANT_DESA_BAD)).sum())

Total: 144
Deceased 106
Living 38


### Number of DESA per group [total 313]

In [13]:
from collections import defaultdict
from src.constants import Epitope_DB

cls_vs_desa = defaultdict(set)
for ep in desas: 
    for group, epitopes in Epitope_DB.items():
        if ep in epitopes: 
            cls_vs_desa[group].add(ep)
{cls: len(_set) for cls, _set in cls_vs_desa.items()}

{'DR': 89, 'ABC': 158, 'DQ': 57}

### Relevant DESA per group

In [14]:
cls_vs_desa = defaultdict(set)
for ep in RELEVANT_DESA_BAD: 
    for group, epitopes in Epitope_DB.items():
        if ep in epitopes: 
            cls_vs_desa[group].add(ep)

In [24]:
print(dict(cls_vs_desa))

{'DQ': {'45EV', '45GV', '67VG'}, 'ABC': {'80TA', '45KE', '62QE', '80TLR', '66NH', '158T', '71TD', '144QL', '163RG', '76ET', '150AAH', '76ED', '44KM', '114R'}, 'DR': {'71A', '70DRA', '67F', '70DA'}}


In [16]:
{
    'ABC': {'80TA', '45KE', '62QE', '80TLR', '66NH', '158T', '71TD', '144QL', '163RG', '76ET', '150AAH', '76ED', '44KM', '114R'}, 
    'DQ': {'45EV', '45GV', '67VG'}, 
    'DR': {'71A', '70DRA', '67F', '70DA'},
}

{'ABC': {'144QL', '158T', '71TD', '76ED', '76ET', '80TLR'},
 'DR': {'67F', '70DA', '70DRA'},
 'DQ': {'45EV'}}

### number of transplants with clinically relevant DESA:

In [17]:
df[df.DESA_Status.eq('DESA')]['DESA'].apply(lambda x: len(x & RELEVANT_DESA_BAD)).value_counts()

0    295
1     95
2     18
3     13
4     11
5      4
6      3
Name: DESA, dtype: int64

### number of transplants directed to HLA

In [23]:
get_rel_hla = lambda x: {hla.split('*')[0] for ep, hla in x.items() if ep in RELEVANT_DESA_BAD}
df[(df.DESA_Status == 'DESA')]['EpvsHLA_Donor'].apply(get_rel_hla).value_counts()

{}                    295
{DQB1}                 50
{B}                    42
{A}                    28
{DRB1}                  6
{A, B}                  6
{DRB5, DRB1}            4
{DQB1, DRB1}            2
{A, DQB1}               2
{DRB5}                  1
{DRB1, B}               1
{DQB1, DRB1, DRB5}      1
{DQB1, B}               1
Name: EpvsHLA_Donor, dtype: int64

### Tineke Follow-up question: I also need detailed information about all DESAs (loci, number per patient).

In [22]:
get_hla = lambda x: {hla.split('*')[0] for ep, hla in x.items()}
hla_vs_desa = df[df.DESA_Status == 'DESA']['EpvsHLA_Donor'].apply(get_hla).value_counts().sort_values(ascending=False)
hla_vs_desa

{A}                         83
{B}                         80
{DQB1}                      57
{DRB1}                      36
{A, B}                      20
{C}                         17
{DRB3}                      15
{DRB1, DRB3}                11
{DRB4}                       9
{B, C}                       8
{DQB1, DRB1}                 7
{A, DRB1}                    6
{A, C}                       6
{DRB1, B}                    6
{DRB5}                       5
{DRB4, DRB1}                 5
{DRB5, DRB1}                 5
{DQB1, DQA1}                 5
{A, DQB1}                    4
{A, DRB1, B}                 4
{DQB1, C}                    3
{DQB1, DRB4}                 3
{DQB1, DRB3}                 2
{DQB1, DRB1, DQA1}           2
{A, B, C}                    2
{A, DQB1, DQA1}              2
{A, DRB1, DRB3}              2
{A, DRB1, DQB1}              2
{DQB1, DRB1, DRB3}           2
{A, DRB4, DRB1}              1
{DRB5, DRB1, B, C}           1
{DQB1, DRB1, DRB5, DRB3}     1
{DRB5, B